In [1]:
suppressWarnings(suppressPackageStartupMessages({
    library(tidyverse)
    library(data.table)
}))


In [2]:
# input
PRS_d='/scratch/groups/mrivas/projects/PRS/private_output/202009_batch'
covar_f='/oak/stanford/groups/mrivas/ukbb24983/sqc/population_stratification_w24983_20200828/ukb24983_GWAS_covar.20200828.phe'
PRS_test_run_f='/oak/stanford/groups/mrivas/projects/PRS/private_output/20200908_PRS_map_test/7_pfactor_v5/%s/2_refit/%s.sscore.zst'

# output
master_PRS_f=file.path(PRS_d, 'ukb24983_GWAS_covar.20200828.PRSs.phe')


In [13]:
source('/oak/stanford/groups/mrivas/users/ytanigaw/repos/rivas-lab/snpnet/helpers/snpnet_misc.R')


In [9]:
system(paste('find', PRS_d, '-maxdepth 2 -name "snpnet.2_refit.PRSs.*.phe" | sort -V'), intern=T) %>%
lapply(function(f){
    fread(f, colClasses = c('#FID'='character', 'IID'='character')) %>%
    rename('FID'='#FID')
}) %>% reduce(function(x, y){inner_join(x, y, by=c('FID', 'IID'))}) -> PRS_df


In [10]:
PRS_df %>% dim()

[1] 488377   1770

In [11]:
covar_f %>% fread(colClasses = c('#FID'='character', 'IID'='character')) %>%
rename('FID'='#FID') -> covar_df


In [14]:
c('INI50', 'INI21001', 'HC269', 'HC382') %>%
lapply(function(gbeid){
    read_PRS(sprintf(PRS_test_run_f, gbeid, gbeid)) %>% rename(!!sprintf('PRS_%s', gbeid) := 'geno_score')
}) %>% reduce(function(x, y){inner_join(x, y, by=c('FID', 'IID'))}) -> PRS_test_run_df


In [15]:
covar_df %>% 
left_join(PRS_test_run_df, by=c('FID', 'IID')) %>%
left_join(PRS_df, by=c('FID', 'IID')) -> full_df


In [16]:
full_df %>% dim()

[1] 488377   1866

In [17]:
full_df %>%
rename('#FID' = 'FID') %>%
fwrite(master_PRS_f, sep='\t', na = "NA", quote=F)
